# Generador de preguntas

Debemos construir un dataset de preguntas para la evaluación de nuestro RAG

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from langchain_community.document_loaders import PyPDFDirectoryLoader

from tqdm import tqdm
import time
import warnings
from dotenv import load_dotenv
import os

Cargamos nuestra key de mistralai, necesaria para usar sus modelos

In [2]:
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
warnings.filterwarnings("ignore", category=FutureWarning)

Usaremos el modelo mistral-small, que es gratuito y funciona bastante bien

In [3]:
llm = ChatMistralAI(
    model="mistral-small-latest", 
    mistral_api_key=MISTRAL_API_KEY,
    temperature=0.1, 
    random_seed=12345
)

Prompt que se encargará de generar el par pregunta-respuesta

In [4]:
QA_generation_prompt = """
Tu tarea es generar una ÚNICA pregunta con su respectiva respuesta basándote únicamente en el contexto aportado.
Tus preguntas deben estar relacionadas con la DANA (Depresión Aislada en Niveles Altos) ocurrida a finales de 2024.
Las preguntas generadas deben tener un formato de preguntas frecuentas (FAQ).
Debes formular tus preguntas imitando el estilo que usaría una persona afectada por la catástrofe buscando información.
Tus preguntas deben poder ser respondidas usando una pieza específica del contexto.
No debes mencionar cosas como "de acuerdo a la información" o "según el contexto".
Escribe en ESPAÑOL.
---
Realiza tu respuestas como sigue, manteniendo un formato separado por comas (csv):

"tu pregunta","tu respuesta"

---
Contexto: {context}\n
Output:::"""

Cargamos los documentos y generamos la chain

In [5]:
documents = PyPDFDirectoryLoader("../my_data").load()
prompt = ChatPromptTemplate.from_template(QA_generation_prompt)
chain =  prompt | llm | StrOutputParser()

Creamos el fichero y ejecutamos un bucle para obtener nuestro dataset completo

In [8]:
with open("preguntas_mistral_plus2.csv", "w", encoding="utf-8") as f:
    f.write("Pregunta, Respuesta, Contexto, Recurso\n")
    for doc in tqdm(documents, desc="Generando hoja de preguntas", unit="páginas"):
        page_content = doc.page_content.replace('\n', ' ').replace('"', '""')
        response = chain.invoke({"context":page_content})
        source = doc.metadata["source"]
        line = f'{response},"{page_content}","{source}"' 
        f.write(line + "\n")
        time.sleep(2)   # to avoid rate limit issues

Generando hoja de preguntas: 100%|██████████| 569/569 [1:09:40<00:00,  7.35s/páginas]
